In [7]:
import nltk

from nltk.corpus import PlaintextCorpusReader
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk import sent_tokenize
import string
import re 


In [8]:
# instantiate finBERT
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [9]:
import os

# loop through text file in Data
rootdir = 'Data'
filelist = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filename = os.path.join(subdir, file)
        filelist.append(filename)

filelist = filelist[1:]
# to be changed to the full list
print(filelist)

['Data/BA_transcripts/2016-Jan-28-BA.N-138020839119-Transcript.txt', 'Data/BA_transcripts/2015-Apr-22-BA.N-140063175207-Transcript.txt', 'Data/BA_transcripts/2022-Oct-26-BA.N-140697322944-Transcript.txt', 'Data/BA_transcripts/2010-Jul-28-BA.N-137028827968-Transcript.txt', 'Data/BA_transcripts/2014-Jan-29-BA.N-139067099110-Transcript.txt', 'Data/BA_transcripts/2016-Apr-27-BA.N-136945469455-Transcript.txt', 'Data/BA_transcripts/2013-Oct-23-BA.N-137926414670-Transcript.txt', 'Data/BA_transcripts/2018-Oct-24-BA.N-141011416206-Transcript.txt', 'Data/BA_transcripts/2012-Jul-25-BA.N-137340637566-Transcript.txt', 'Data/BA_transcripts/2017-Oct-25-BA.N-137402596339-Transcript.txt', 'Data/BA_transcripts/2016-Oct-26-BA.N-138181083067-Transcript.txt', 'Data/BA_transcripts/2021-Jul-28-BA.N-139176649402-Transcript.txt', 'Data/BA_transcripts/2020-Jul-29-BA.N-139449958669-Transcript.txt', 'Data/BA_transcripts/2017-Jan-25-BA.N-137214911074-Transcript.txt', 'Data/BA_transcripts/2013-Jul-24-BA.N-138218065

In [10]:
def textPreprocessing(filename):
    # load text
    file = open(filename, 'rt')
    text = file.read()
    file.close()

    # tokenize into sentences
    tokens = sent_tokenize(text)

    # alpha to lowercase
    tokens = [w.lower() for w in tokens]

    # remove punctuation and whitespaces
    remove = string.punctuation
    remove = re.sub(r"[-]+", "", remove)
    pattern = r"[{}]".format(remove + '-')
    clean_list = []

    for sentence in tokens:
        sentence = re.sub(pattern, "", sentence) 
        sentence = sentence.split()
        sentence = ' '.join(sentence)
        clean_list.append(sentence)

    return clean_list

# no neeed to remove stopwords and conduct lemmatization
# may worsen prediction for transformer


In [11]:
# function that takes returns a document level sentiment score

def sentimentAnalysis(filename):
    clean_list = textPreprocessing(filename)
    # pipeline: API that wraps finBERT sentiment analysis 
    nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)
    results = nlp(clean_list)
    
    negative = 0
    positive = 0
    for element in results:
        if element['label'] == "Negative":
            negative += 1
        elif element['label'] == "Positive":
            positive += 1
    total = len(results)

    sentiment_score = (positive - negative)/ total
    return sentiment_score

    # +ve sentiment score: positive earnings call
    # -ve sentiment score: negative earnings call

    # https://www.alpha-sense.com/blog/engineering/sentiment-score/
    # sentiment score: (positive - negative) / total number of statements





In [12]:
sentiment_score_dict = {'filename':[], "sentiment_score": []}
for filename in filelist:
    sentiment_score = sentimentAnalysis(filename)
    filename = filename.split('/')[-1]
    sentiment_score_dict["filename"].append(filename)
    sentiment_score_dict["sentiment_score"].append(sentiment_score)

print(sentiment_score_dict)


{'filename': ['2016-Jan-28-BA.N-138020839119-Transcript.txt', '2015-Apr-22-BA.N-140063175207-Transcript.txt', '2022-Oct-26-BA.N-140697322944-Transcript.txt', '2010-Jul-28-BA.N-137028827968-Transcript.txt', '2014-Jan-29-BA.N-139067099110-Transcript.txt', '2016-Apr-27-BA.N-136945469455-Transcript.txt', '2013-Oct-23-BA.N-137926414670-Transcript.txt', '2018-Oct-24-BA.N-141011416206-Transcript.txt', '2012-Jul-25-BA.N-137340637566-Transcript.txt', '2017-Oct-25-BA.N-137402596339-Transcript.txt', '2016-Oct-26-BA.N-138181083067-Transcript.txt', '2021-Jul-28-BA.N-139176649402-Transcript.txt', '2020-Jul-29-BA.N-139449958669-Transcript.txt', '2017-Jan-25-BA.N-137214911074-Transcript.txt', '2013-Jul-24-BA.N-138218065365-Transcript.txt', '2020-Apr-29-BA.N-138817718487-Transcript.txt', '2011-Oct-26-BA.N-137584057258-Transcript.txt', '2022-Apr-27-BA.N-137142942545-Transcript.txt', '2021-Apr-28-BA.N-137047951222-Transcript.txt', '2015-Oct-21-BA.N-137198648580-Transcript.txt', '2023-Jan-25-BA.N-13934222

In [13]:
df = pd.DataFrame(sentiment_score_dict)

df.to_excel('sentiment_score.xlsx', index=False)

In [14]:
# # stopword removal
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))
# words = [w for w in words if not w in stop_words]
# print(words[:100])

In [15]:
# # lemmatization
# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()
# lemmatized = [lemmatizer.lemmatize(word) for word in words]
# print(lemmatized) 